In [1]:
!python --version

Python 3.7.6


In [3]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [4]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [5]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [6]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [7]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [9]:
ws = WS('C:/Users/zero0/OneDrive/cupoy/NLP/Day007/data/')
pos = POS('C:/Users/zero0/OneDrive/cupoy/NLP/Day007/data/')

In [11]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title], sentence_segmentation=True)
    word_p = pos(word_s)
    # END YOUR CODE
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [12]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title], sentence_segmentation=True)
    word_p = pos(word_s)
    # END YOUR CODE
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [13]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# Bag of Words (BOW)

In [14]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
# YOUR CODE HERE
idx = 0
for word_pos_list in train_title_cuts.values():
    for word, _ in word_pos_list:
        if word not in word2index:
            word2index[word] = idx
            index2word[idx] = word
            idx += 1
# END YOUR CODE

In [15]:
word2index['溫暖']

1512

In [16]:
index2word[1520]

'許'

In [17]:
def get_bow_vector(pairs, word2index):
    # YOUR CODE HERE
    vector = np.zeros(len(word2index))
    for word, _ in pairs:
        if word in word2index:
            vector[word2index[word]] += 1 
    # END YOUR CODE
    return vector

In [18]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 1., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [19]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['帕克成', '李秋平', '卡納瓦羅', '皮爾洛', '·瞻', '魅族', '谷歌', '意大利杯', '德甲', '楊智復', '巔峰賽', '海富通', '東皇', '米切爾西蒙斯', '李白界', '國六', '皇馬', '殺里', '神帥', '特里', '博時', '星巴克', '海爾', '布茲德里克', '慧商', '斐濟', '庫里', '安永', '朱嘯虎', '紫鑫', '美宇', '烏迪內斯', '陳柯楓', '德帥', '德羅', '張大仙', '許', '張某', '堪比比薩', '哈登', '耀里', '騰訊王', '京新', '英特爾', '阿里雲', '嬴政', '拳皇', '亞亞·圖雷', '凱西', '克羅托內 ', '埃梅里', '周雲傑', '鍾馗', '胤祥', '羅永浩', '杜蘭特', '王昭君', '李白', '埃爾克森', '王者榮', '張呈棟', '纏中', '卓爾', '桑德羅', '維亞利', '韓寒', '德羅贊', '埃姆雷', '萬孚', '峽谷里', '舒斯特爾', '張藝謀', '何享健', '約翰塞納', '奧恰洛夫', '大豐', '詹姆斯', '尤文', '周評', '西蒙斯', '張掖篇', '拜仁', '安琪', '姆巴佩', '微博', '遼足', '2018世界杯', '新游', '余則成', '維特爾', '海王星', '杜蕾斯', '張本智和', '雷霆隊', '施之皓', '海因克斯 萊萬多夫斯基', '弗格森', '阿珂', '樊振東', '小智']
Na : ['大四', '報業', '力量', '帝', '素能', '新招', '盛贊', '戰績', '贏家', '爪子', '國腳', '心理', '特幣', '正負值', '龍頭', '柑橘', '鈣', '墒情', '紫薇', '國安', '排他性', '報告', '網友數', '海', '素顏', '挑戰', '樹莓', '茅台', '印度人', '理想', '籌股', '需求', '蛋', '兄弟', '樹木', '設計', '猴子', '仇', '魂牛', '路', '蜂農', '插秧期', '股票池', '生態', '龍蝦', '豬價', '規範', '兩護法', '淚', '博物', '稀罕物

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [20]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        # YOUR CODE HERE
        'Caa', 'Cab','Cba', 'Cbb', 'Di', 'Dk', 'DM', 'I'
        # END YOUR CODE
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [23]:
def cosine_similarity(bow1, bow2):
    # YOUR CODE HERE
    len_bow1 = (bow1 **2).sum() **(1/2)
    len_bow2 = (bow2 **2).sum() **(1/2)

    similarity = np.sum(bow1*bow2) /(len_bow1*len_bow2)
    # END YOUR CODE
    return similarity

In [24]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [25]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [26]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [27]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0.04      , 0.00333333, 0.15666667, ..., 0.        , 0.        ,
        0.        ]),
 '財經': array([0.        , 0.        , 0.12333333, ..., 0.        , 0.        ,
        0.        ]),
 '科技': array([0.        , 0.        , 0.07666667, ..., 0.        , 0.        ,
        0.        ]),
 '旅遊': array([0.        , 0.        , 0.08666667, ..., 0.        , 0.        ,
        0.        ]),
 '農業': array([0.        , 0.        , 0.09333333, ..., 0.        , 0.        ,
        0.        ]),
 '遊戲': array([0.        , 0.        , 0.27      , ..., 0.00333333, 0.00333333,
        0.00333333])}

# Group mean vector: 測試

In [28]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [29]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 55, '財經': 16, '農業': 12, '旅遊': 11, '科技': 10, '遊戲': 8})
predict 財經 :  Counter({'財經': 26, '體育': 6, '科技': 6, '農業': 5, '旅遊': 4, '遊戲': 3})
predict 科技 :  Counter({'科技': 60, '旅遊': 29, '財經': 26, '農業': 23, '遊戲': 22, '體育': 18})
predict 旅遊 :  Counter({'旅遊': 41, '財經': 13, '農業': 12, '遊戲': 10, '體育': 4, '科技': 4})
predict 農業 :  Counter({'農業': 44, '科技': 14, '財經': 9, '旅遊': 9, '遊戲': 9, '體育': 8})
predict 遊戲 :  Counter({'遊戲': 48, '財經': 10, '體育': 8, '旅遊': 6, '科技': 5, '農業': 4})
